**File Format**:

- Pipe-delimited (`|`) format
- **Non-UTF-8 encoding** (you'll need to handle encoding issues)
- Contains **data quality issues**:
    - Some fields have comma-separated values within them
    - Some rows may have missing or extra fields
    - Some numeric values may have formatting issues (commas in numbers)
    - Some records have invalid data (zero quantities, negative prices, wrong ID formats)

#### Read Sales Data with Encoding Handling

In [1]:
import csv
from pathlib import Path

In [2]:
def read_sales_data(filename, file_encoder):
    data = []
    try:
        with open(filename, mode='r', encoding=file_encoder, newline='\n') as file:
            file_content = csv.reader(file, delimiter='|')
            header = next(file_content, None)

            for row in file_content:
                if row and any(field.strip() for field in row):
                    data.append('|'.join(row))
        return data

    except UnicodeDecodeError:
        print(f'{filename} file is not in UTF-8 encoding')
        return data
    except FileNotFoundError:
        print(f'{filename} file does not exist.')
        return data

In [3]:
BASE_DIR = Path.cwd().parent   # sales-analytics-system
file_path = BASE_DIR / 'data' / 'sales_data.txt'

In [4]:
a = read_sales_data(file_path,'utf-8')

In [5]:
first_row = a[0].split('|')

In [6]:
first_row

['T018', '2024-12-29', 'P107', 'USB Cable', '8', '173', 'C009', 'South']

In [7]:
def parse_transactions(raw_line):

    data = []
    for line in raw_line:
        t_id, dt, p_id, p_name, qty_raw, price_raw, c_id, region = [f.strip() for f in line.split('|')]
        
        # Handle commas within ProductName (replace commas with space)
        p_name_clean = p_name.replace(",", " ").strip()

        # Remove commas from numeric fields (e.g., "45,000")
        qty_clean = qty_raw.replace(",", "").strip()
        price_clean = price_raw.replace(",", "").strip()

        try:
            qty = int(qty_clean)
            unit_price = float(price_clean)
        except ValueError:
            continue

        data.append(
            {
                "TransactionID": t_id,
                "Date": dt,
                "ProductID": p_id,
                "ProductName": p_name_clean,
                "Quantity": qty,
                "UnitPrice": unit_price,
                "CustomerID": c_id,
                "Region": region,
            }
        )

    return data

In [8]:
a

['T018|2024-12-29|P107|USB Cable|8|173|C009|South',
 'T063|2024-12-07|P110|Laptop Charger|6|1,916|C022|East',
 'T075|2024-12-10|P106|Headphones|0|2826|C001|South',
 'T023|2024-12-09|P109|Wireless Mouse|9|523|C022|North',
 'T059|2024-12-29|P102|Mouse,Wireless|4|1056|C010|South',
 'T035|2024-12-08|P102|Mouse|4|431|C011|North',
 'T061|2024-12-10|P109|Wireless Mouse|2|775|C009|North',
 'T057|2024-12-15|P101|Laptop,Premium|10|81896|C004|North',
 'T034|2024-12-22|P107|USB Cable|6|324|C029|West',
 'T050|2024-12-02|P104|Monitor,LED|10|9997|C024|East',
 'T024|2024-12-25|P109|Wireless Mouse|5|1812|C011|North',
 'T004|2024-12-07|P109|Wireless Mouse|9|1359|C008|West',
 'T068|2024-12-02|P109|Wireless Mouse|6|1,692|C018|South',
 'T066|2024-12-06|P105|Webcam|8|4,259|C023|West',
 'T064|2024-12-16|P109|Wireless Mouse|5|604|C003|West',
 'T045|2024-12-26|P108|External Hard Drive|9|3802|C002|North',
 'T015|2024-12-30|P105|Webcam|9|2899|C022|East',
 'T055|2024-12-07|P105|Webcam,HD|6|2977|C009|West',
 'T072

In [9]:
def validate_and_filter(transactions, region=None, min_amount=None, max_amount=None):
    """
    Validates transactions and applies optional filters
    """
    required_fields = [
        "TransactionID", "Date", "ProductID", "ProductName",
        "Quantity", "UnitPrice", "CustomerID", "Region"
    ]

    total_input = len(transactions)
    invalid_count = 0
    valid_transactions = []

    # --- Print available regions (from all input, if present) ---
    regions = sorted({
        t.get("Region", "").strip()
        for t in transactions
        if isinstance(t, dict) and t.get("Region")
    })
    print("Available regions:", regions if regions else "None found")

    # --- Validate transactions ---
    for txn in transactions:
        # Must be a dict
        if not isinstance(txn, dict):
            invalid_count += 1
            continue

        # All required fields must exist and be non-empty (basic check)
        missing = [k for k in required_fields if k not in txn or txn[k] in (None, "")]
        if missing:
            invalid_count += 1
            continue

        # ID prefix rules
        if not str(txn["TransactionID"]).startswith("T"):
            invalid_count += 1
            continue
        if not str(txn["ProductID"]).startswith("P"):
            invalid_count += 1
            continue
        if not str(txn["CustomerID"]).startswith("C"):
            invalid_count += 1
            continue

        # Quantity and UnitPrice positive + type-safe
        try:
            qty = int(txn["Quantity"])
            price = float(txn["UnitPrice"])
        except (ValueError, TypeError):
            invalid_count += 1
            continue

        if qty <= 0 or price <= 0:
            invalid_count += 1
            continue

        # Store normalized numeric values back (optional but helpful)
        txn["Quantity"] = qty
        txn["UnitPrice"] = price

        valid_transactions.append(txn)

    # --- Amount range print (computed from valid transactions) ---
    if valid_transactions:
        amounts = [t["Quantity"] * t["UnitPrice"] for t in valid_transactions]
        print(f"Transaction amount range (valid only): min={min(amounts):.2f}, max={max(amounts):.2f}")
    else:
        print("Transaction amount range: no valid transactions to compute range.")

    # Summary counters
    filtered_by_region = 0
    filtered_by_amount = 0

    current = valid_transactions
    print(f"After validation: {len(current)} records (invalid: {invalid_count})")

    # --- Region filter ---
    if region is not None:
        before = len(current)
        current = [t for t in current if str(t.get("Region", "")).strip().lower() == str(region).strip().lower()]
        filtered_by_region = before - len(current)
        print(f"After region filter ({region}): {len(current)} records")

    # --- Amount filters ---
    # Compute amounts once for filtering
    def amount(t):
        return t["Quantity"] * t["UnitPrice"]

    if min_amount is not None:
        before = len(current)
        current = [t for t in current if amount(t) >= float(min_amount)]
        filtered_by_amount += before - len(current)
        print(f"After min_amount filter ({min_amount}): {len(current)} records")

    if max_amount is not None:
        before = len(current)
        current = [t for t in current if amount(t) <= float(max_amount)]
        filtered_by_amount += before - len(current)
        print(f"After max_amount filter ({max_amount}): {len(current)} records")

    filter_summary = {
        "total_input": total_input,
        "invalid": invalid_count,
        "valid_records": len(valid_transactions),
        "filtered_by_region": filtered_by_region,
        "filtered_by_amount": filtered_by_amount,
        "final_count": len(current)
    }

    return current, filter_summary

In [10]:
first_row = parse_transactions(a)

In [11]:
clean_data = parse_transactions(a)

In [12]:
clean_data, summary_data = validate_and_filter(clean_data,'North', 300, 5000)

Available regions: ['East', 'North', 'South', 'West']
Transaction amount range (valid only): min=257.00, max=818960.00
After validation: 70 records (invalid: 10)
After region filter (North): 21 records
After min_amount filter (300): 20 records
After max_amount filter (5000): 7 records


In [13]:
summary_data

{'total_input': 80,
 'invalid': 10,
 'valid_records': 70,
 'filtered_by_region': 49,
 'filtered_by_amount': 14,
 'final_count': 7}

In [14]:
clean_data

[{'TransactionID': 'T023',
  'Date': '2024-12-09',
  'ProductID': 'P109',
  'ProductName': 'Wireless Mouse',
  'Quantity': 9,
  'UnitPrice': 523.0,
  'CustomerID': 'C022',
  'Region': 'North'},
 {'TransactionID': 'T035',
  'Date': '2024-12-08',
  'ProductID': 'P102',
  'ProductName': 'Mouse',
  'Quantity': 4,
  'UnitPrice': 431.0,
  'CustomerID': 'C011',
  'Region': 'North'},
 {'TransactionID': 'T061',
  'Date': '2024-12-10',
  'ProductID': 'P109',
  'ProductName': 'Wireless Mouse',
  'Quantity': 2,
  'UnitPrice': 775.0,
  'CustomerID': 'C009',
  'Region': 'North'},
 {'TransactionID': 'T008',
  'Date': '2024-12-09',
  'ProductID': 'P110',
  'ProductName': 'Laptop Charger',
  'Quantity': 1,
  'UnitPrice': 2994.0,
  'CustomerID': 'C015',
  'Region': 'North'},
 {'TransactionID': 'T030',
  'Date': '2024-12-08',
  'ProductID': 'P105',
  'ProductName': 'Webcam',
  'Quantity': 1,
  'UnitPrice': 2986.0,
  'CustomerID': 'C029',
  'Region': 'North'},
 {'TransactionID': 'T037',
  'Date': '2024-12